## Atividade 01 - 12/10/2023

### **Importando as bibliotecas**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### **Usando o Pandas para ler meu arquivo de dados .csv e criar um dataframe**

In [ ]:
#Declaração da variável 'df', que vai receber meu arquivo .csv com os dados brutos
#Usando header=[0] pra dizer que a primeira linha do código é índice, e index_col=[0] pra dizer que a primeira coluna é índice
df = pd.read_csv('C:/Users/bruna/Documents/ENS410064/ENS410064/dados/brutos/abundancia_especies_benthos.csv', header=[0], index_col=[0])     

#Calculando a média de individuos para cada espécie, para todo o dataframe, considerando axis = 1, que significa
#que eu quero a média entre os valores das colunas para cada linha
media_especie = df.mean(axis=1).round(2) #Arredonda o resultado para duas casas decimais
print(media_especie)


In [ ]:
#Criando uma nova coluna chamada 'media', para receber os valores da média de indivíduos de cada espécie
df=df.assign(media = media_especie)
df

In [ ]:
#Salvando minha base agora com as médias
df.to_csv(r'C:\Users\bruna\Documents\ENS410064\ENS410064\dados\brutos\abundancia_benthos_com_media.csv')

### **Algumas análises** ###

In [ ]:
#Quero descobrir quais espécies apresentaram média maior do que 100, e quais espécies apresentaram média inferior a 10

media_100 = df[(df['media']>=100)]
print(media_100.index.tolist(), '\n') #Transforma o índice em lista e depois printa os nomes das espécies que tiveram média >100
print(f'A quantidade de espécies com média de indivíduos superior a 100 é de: {len(media_100)}')

In [ ]:
media_10 = df[(df['media']<=10)]
print(media_10.index.tolist(), '\n')
print(f'A quantidade de espécies com média de indivíduos inferior a 10 é de: {len(media_10)}')

In [ ]:
#Quantos e quais indivíduos apresentaram média inferior a 50

media_50 = df[(df['media']<=50)]
print(media_50.index.tolist(), '\n')

print(f'A quantidade de espécies com média de indivíduos inferior a 50 é de: {len(media_50)}')

In [ ]:
#Quero ver qual foi a espécie com a maior e a menor média

maior_media = df[(df['media'] == df['media'].max())]
menor_media = df[(df['media'] == df['media'].min())]

print(f'A espécie com a maior média de indivíduos foi: {maior_media.index.tolist()}''\n')
print(f'A espécie com a menor média de indivíduos foi: {menor_media.index.tolist()}')

### **Deletando dados das espécies cuja média de indivíduos entre 1972 e 2012 foi inferior a 50**

In [ ]:
#Como muitas espécies quase não tiveram indivíduos identificados em quantidades significativas, optei por
#excluir uma parte deles para trabalhar somente com aqueles que apareceram mais

df = df.drop(df[(df['media'] <=50)].index) #O comando 'drop' serve para excluir linhas ou colunas. Nesse caso, exclui as linhas das espécies cuja média de indivíduos ('media', a última coluna que foi criada) foi inferior ou igual a 50
df.shape
df

#### **Gerando gráficos**

#### **Gráficos da evolução de cada espécie ao longo do tempo**

In [ ]:
#Tentando plotar todos os dados
#df.plot
#O resultado vai ficar melhor se a matriz for transposta, pois quero as espécies como head da tabela e cada data em uma linha (index)
df_t = df.transpose()  

#Usando um laço de repetição para não precisar gerar um gráfico de cada vez
for coluna in df_t.columns: #iterando sobre cada coluna (cada coluna é uma espécie)
    especie = coluna #Aqui, estou declarando uma variável chamada 'espécie' e atribuindo a ela o nome que está na coluna atualmente iterada do meu dataframe
    plt.figure(figsize=(10,8))
    plt.plot(df_t.index, df_t[coluna],color='green') #plota no eixo x o índice (que são as datas), e no eixo y a quantidade de indivíduos para aquela especie definida na coluna
    plt.xlabel('Season/year')
    plt.ylabel('Number of organisms')
    plt.title(f'Evolution of {especie} over time') #esse 'f' na frente permite que seja inserida a variável dentro do texto
    plt.xticks(df_t.index[::5], rotation=45)
    plt.tight_layout()  # Evita que as legendas se sobreponham
    nome_arquivo=f'C:/Users/bruna/Documents/ENS410064/ENS410064/dados/graficos/{especie}_evolution.png'
    plt.savefig(nome_arquivo) #Essa função permite salvar a figura 
    plt.show()
    

        

#### **Gráfico da evolução de todas as espécies ao longo do tempo**

In [ ]:
plt.figure(figsize=(25,15)) #definindo o tamanho da minha figura, que nesse caso precisa ser grande pois são muitos dados sobrepostos

for coluna in df_t.columns: #iterando sobre todas as colunas do meu df transposto
    plt.plot(df_t.index, df_t[coluna], label=coluna) #definindo os eixos do gráfico: x = meu índice, que é o tempo; y = minhas colunas, que são as espécies, label é o rótulo de cada linha

plt.xlabel('Season/year', fontsize="20")
plt.ylabel('Number of organisms', fontsize="20")
plt.title('Evolution of all benthic species over time', fontsize="30")
plt.legend(fontsize="15", loc='best') #Essa função loc='best' posiciona a legenda (no caso, com as espécies) no melhor local possível do gráfico
plt.xticks(df_t.index[::5], fontsize="20", rotation=45)
plt.yticks(fontsize="20")
plt.tight_layout()
nome_arquivo='C:/Users/bruna/Documents/ENS410064/ENS410064/dados/graficos/evolution_all_species.png'
plt.savefig(nome_arquivo) 
plt.show()


#### **Gráficos para grupos de espécies**

In [ ]:
#Tenho 24 espécies e quero seis gráficos:
max_esp_por_subplot = 4 #Aqui estou definindo que o máximo de espécies que pode sair em cada gráfico é 4. 

# Criando subplots automaticamente com base no número máximo de espécies por subplot
num_subplots = len(df_t.columns) // max_esp_por_subplot 
#Na linha acima, a função 'len' me dá o número de colunas que temos, que no caso, é o número de espécies,
#e divide o número de espécies pelo máximo de espécies em cada gráfico. Assim, temos 24/4 = 6 gráficos (subplots)

#Criando um laço de repetição, que define os grupos de espécies
#O índice i e variável grupo_especies estão recebendo o número do grupo (grupo 1, 2, 3, 4...)
#Para cada grupo, está sendo gerado um subplot de tamanho 10x5
for i, grupo_especies in enumerate(range(0, len(df_t.columns), max_esp_por_subplot)): #começa em zero, vai até o nº colunas, pulando de 4 em 4
    plt.figure(figsize=(10, 5))  

#Esse laço de repetição itera sobre as colunas do df_t, começando em grupo_especies, que tem o índice do grupo (1, 2, 3... 6),
#e indo até o final do grupo, que nada mais é do que somar o máximo número de espécies por grupo
#pra cada grupo, está sendo plotado um gráfico de tempo x espécie (coluna), com a legenda sendo as espécies
    for coluna in df_t.columns[grupo_especies:grupo_especies+max_esp_por_subplot]:
        plt.plot(df_t.index, df_t[coluna], label=coluna)
        
#Definições de formatação de gráfico e local para salvar as figuras    
    plt.xlabel('Season/year')
    plt.ylabel('Number of organisms')
    plt.title(f'Evolução das Espécies (Grupo {i+1})', fontsize="15")
    plt.legend(loc='best')
    plt.tight_layout()
    plt.xticks(df_t.index[::5], rotation=45, fontsize="10")
    plt.yticks(fontsize="10")
    nome_arquivo=f'C:/Users/bruna/Documents/ENS410064/ENS410064/dados/graficos/evolution_species_group_{i+1}.png'
    plt.savefig(nome_arquivo) 

plt.show()